# TCP Server Implementation

> Building a TCP server with detailed explanations of each step

In [ ]:
#| default_exp server

## Introduction

In this notebook, we'll implement a TCP server that:
1. Creates a socket
2. Binds to an address
3. Listens for incoming connections
4. Accepts connections and handles them
5. Sends and receives data
6. Closes connections gracefully

Let's import the necessary modules:

In [ ]:
#| export
from python_tcp.core import *
import socket
from typing import Optional, List, Tuple, Dict, Any, Union, Callable
import threading
import time
import uuid

## Simple TCP Server

We'll start with a basic TCP server implementation that will handle connections one at a time:

In [ ]:
#| export
class TCPServer:
    """A simple TCP server that can handle multiple clients."""
    
    def __init__(self, host: str = LOCALHOST, port: int = 0, 
                 backlog: int = DEFAULT_BACKLOG,
                 buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the server with host, port, and other parameters.
        
        If port is 0, a random available port will be assigned.
        """
        self.host = host
        self.port = port if port != 0 else get_free_port()
        self.backlog = backlog
        self.buffer_size = buffer_size
        self.sock = None
        self.state = SocketState.CLOSED
        self.connections: Dict[str, TCPConnection] = {}
        self.running = False
        self.accept_thread = None
        
    def __str__(self) -> str:
        """String representation of the server."""
        return f"TCPServer at {self.host}:{self.port} (state: {self.state})"
    
    def start(self) -> None:
        """Start the server: create socket, bind, and begin listening."""
        if self.sock:
            print("Server already started")
            return
            
        # Create a TCP socket
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # Set socket options
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
        # Bind the socket to the address
        self.sock.bind((self.host, self.port))
        
        # Get the actual port (in case 0 was specified)
        if self.port == 0:
            self.port = self.sock.getsockname()[1]
            
        # Start listening for incoming connections
        self.sock.listen(self.backlog)
        self.state = SocketState.LISTEN
        self.running = True
        
        print(f"Server started on {self.host}:{self.port}")
        
        # Start accepting connections in a separate thread
        self.accept_thread = threading.Thread(target=self._accept_connections)
        self.accept_thread.daemon = True
        self.accept_thread.start()
    
    def _accept_connections(self) -> None:
        """Accept incoming connections in a loop."""
        while self.running:
            try:
                # Accept a connection
                client_sock, client_address = self.sock.accept()
                
                # Create a connection ID and store connection info
                conn_id = str(uuid.uuid4())
                connection = TCPConnection(
                    sock=client_sock, 
                    state=SocketState.ESTABLISHED,
                    remote_address=client_address,
                    connection_id=conn_id
                )
                
                self.connections[conn_id] = connection
                
                # Handle client in a new thread
                client_thread = threading.Thread(
                    target=self._handle_client, 
                    args=(connection,)
                )
                client_thread.daemon = True
                client_thread.start()
                
                print(f"New connection from {client_address[0]}:{client_address[1]} (ID: {conn_id})")
            except Exception as e:
                if self.running:  # Only show error if we're supposed to be running
                    print(f"Error accepting connection: {e}")
                break
    
    def _handle_client(self, connection: TCPConnection) -> None:
        """Handle communication with a client."""
        try:
            while self.running and connection.state == SocketState.ESTABLISHED:
                # Receive data from the client
                data = connection.sock.recv(self.buffer_size)
                
                if not data:  # Empty data means the client closed the connection
                    break
                
                # Process the received data (echo it back in this simple example)
                print(f"Received from {connection.connection_id}: {data.decode('utf-8')}")
                connection.sock.sendall(data)
        except Exception as e:
            print(f"Error handling client {connection.connection_id}: {e}")
        finally:
            # Clean up the connection
            self._close_connection(connection)
    
    def send(self, connection_id: str, data: bytes) -> bool:
        """Send data to a specific connection."""
        if connection_id not in self.connections:
            print(f"Connection {connection_id} not found")
            return False
            
        connection = self.connections[connection_id]
        
        try:
            connection.sock.sendall(data)
            return True
        except Exception as e:
            print(f"Error sending data to {connection_id}: {e}")
            self._close_connection(connection)
            return False
    
    def _close_connection(self, connection: TCPConnection) -> None:
        """Close a specific connection."""
        try:
            if connection.sock:
                connection.sock.close()
            
            connection.update_state(SocketState.CLOSED)
            
            if connection.connection_id in self.connections:
                del self.connections[connection.connection_id]
                
            print(f"Connection {connection.connection_id} closed")
        except Exception as e:
            print(f"Error closing connection {connection.connection_id}: {e}")
    
    def stop(self) -> None:
        """Stop the server and close all connections."""
        self.running = False
        
        # Close all client connections
        for conn_id in list(self.connections.keys()):
            self._close_connection(self.connections[conn_id])
        
        # Close the server socket
        if self.sock:
            try:
                self.sock.close()
                print("Server socket closed")
            except Exception as e:
                print(f"Error closing server socket: {e}")
        
        self.sock = None
        self.state = SocketState.CLOSED
        
        # Wait for accept thread to finish
        if self.accept_thread and self.accept_thread.is_alive():
            self.accept_thread.join(timeout=1.0)
            
        print("Server stopped")

## Enhanced TCP Server with Custom Message Handling

Now let's create a more flexible server that allows custom message handling:

In [ ]:
#| export
class EnhancedTCPServer(TCPServer):
    """An enhanced TCP server that supports custom message handlers."""
    
    def __init__(self, host: str = LOCALHOST, port: int = 0, 
                 backlog: int = DEFAULT_BACKLOG,
                 buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the enhanced server."""
        super().__init__(host, port, backlog, buffer_size)
        self.message_handler: Optional[Callable[[str, bytes], Optional[bytes]]] = None
        
    def set_message_handler(self, handler: Callable[[str, bytes], Optional[bytes]]) -> None:
        """Set a custom message handler that will be called when data is received.
        
        The handler should accept connection_id and data parameters, and
        optionally return bytes to send back to the client.
        """
        self.message_handler = handler
    
    def _handle_client(self, connection: TCPConnection) -> None:
        """Override the client handler to use the custom message handler."""
        try:
            while self.running and connection.state == SocketState.ESTABLISHED:
                # Receive data from the client
                data = connection.sock.recv(self.buffer_size)
                
                if not data:  # Empty data means the client closed the connection
                    break
                
                # Process the received data using the custom handler if available
                print(f"Received from {connection.connection_id}: {data.decode('utf-8')}")
                
                if self.message_handler:
                    response = self.message_handler(connection.connection_id, data)
                    if response:
                        connection.sock.sendall(response)
                else:
                    # Default behavior: echo the data back
                    connection.sock.sendall(data)
        except Exception as e:
            print(f"Error handling client {connection.connection_id}: {e}")
        finally:
            # Clean up the connection
            self._close_connection(connection)

## TCP Server with Connection Events

Finally, let's create a version that provides event callbacks for connection events:

In [ ]:
#| export
class EventDrivenTCPServer(EnhancedTCPServer):
    """A TCP server that triggers events for connection lifecycle."""
    
    def __init__(self, host: str = LOCALHOST, port: int = 0, 
                 backlog: int = DEFAULT_BACKLOG,
                 buffer_size: int = DEFAULT_BUFFER_SIZE):
        """Initialize the event-driven server."""
        super().__init__(host, port, backlog, buffer_size)
        self.on_connect: Optional[Callable[[str, Tuple[str, int]], None]] = None
        self.on_disconnect: Optional[Callable[[str], None]] = None
        self.on_data: Optional[Callable[[str, bytes], None]] = None
    
    def _accept_connections(self) -> None:
        """Accept incoming connections and trigger the on_connect event."""
        while self.running:
            try:
                # Accept a connection
                client_sock, client_address = self.sock.accept()
                
                # Create a connection ID and store connection info
                conn_id = str(uuid.uuid4())
                connection = TCPConnection(
                    sock=client_sock, 
                    state=SocketState.ESTABLISHED,
                    remote_address=client_address,
                    connection_id=conn_id
                )
                
                self.connections[conn_id] = connection
                
                # Trigger the on_connect event
                if self.on_connect:
                    try:
                        self.on_connect(conn_id, client_address)
                    except Exception as e:
                        print(f"Error in on_connect callback: {e}")
                
                # Handle client in a new thread
                client_thread = threading.Thread(
                    target=self._handle_client, 
                    args=(connection,)
                )
                client_thread.daemon = True
                client_thread.start()
                
                print(f"New connection from {client_address[0]}:{client_address[1]} (ID: {conn_id})")
            except Exception as e:
                if self.running:  # Only show error if we're supposed to be running
                    print(f"Error accepting connection: {e}")
                break
    
    def _handle_client(self, connection: TCPConnection) -> None:
        """Handle client communication and trigger the on_data event."""
        try:
            while self.running and connection.state == SocketState.ESTABLISHED:
                # Receive data from the client
                data = connection.sock.recv(self.buffer_size)
                
                if not data:  # Empty data means the client closed the connection
                    break
                
                # Trigger the on_data event
                if self.on_data:
                    try:
                        self.on_data(connection.connection_id, data)
                    except Exception as e:
                        print(f"Error in on_data callback: {e}")
                
                # Process the received data using the custom handler if available
                if self.message_handler:
                    response = self.message_handler(connection.connection_id, data)
                    if response:
                        connection.sock.sendall(response)
                else:
                    # Default behavior: echo the data back
                    connection.sock.sendall(data)
        except Exception as e:
            print(f"Error handling client {connection.connection_id}: {e}")
        finally:
            # Clean up the connection
            self._close_connection(connection)
    
    def _close_connection(self, connection: TCPConnection) -> None:
        """Close a connection and trigger the on_disconnect event."""
        conn_id = connection.connection_id
        
        super()._close_connection(connection)
        
        # Trigger the on_disconnect event
        if self.on_disconnect:
            try:
                self.on_disconnect(conn_id)
            except Exception as e:
                print(f"Error in on_disconnect callback: {e}")

## Example Usage

Here's a simple example of how to use our TCP server:

In [ ]:
def simple_server_demo():
    # Create and start a TCP server
    server = TCPServer(port=8000)
    server.start()
    
    try:
        # Keep the server running for 60 seconds
        print("Server running. Press Ctrl+C to stop...")
        time.sleep(60)
    except KeyboardInterrupt:
        print("Keyboard interrupt received, stopping server...")
    finally:
        # Stop the server
        server.stop()

In [ ]:
# Uncomment to run the demo
# simple_server_demo()

In [ ]:
def event_driven_server_demo():
    # Create and start an event-driven TCP server
    server = EventDrivenTCPServer(port=8000)
    
    # Set up event handlers
    server.on_connect = lambda conn_id, addr: print(f"EVENT: Client connected: {addr[0]}:{addr[1]}")
    server.on_disconnect = lambda conn_id: print(f"EVENT: Client disconnected: {conn_id}")
    server.on_data = lambda conn_id, data: print(f"EVENT: Received data from {conn_id}: {data.decode('utf-8')}")
    
    # Set up a custom message handler
    def message_handler(conn_id, data):
        message = data.decode('utf-8')
        # Echo the message back with a prefix
        return f"Server received: {message}".encode('utf-8')
    
    server.set_message_handler(message_handler)
    
    # Start the server
    server.start()
    
    try:
        # Keep the server running for 60 seconds
        print("Event-driven server running. Press Ctrl+C to stop...")
        time.sleep(60)
    except KeyboardInterrupt:
        print("Keyboard interrupt received, stopping server...")
    finally:
        # Stop the server
        server.stop()

In [ ]:
# Uncomment to run the demo
# event_driven_server_demo()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()